## Step 1
### Produce skims
This step is done before running this notebook. Godspeed, lady.

In [1]:
import subprocess

In [ ]:
cmd = "sh scripts/6b_scripts/submit_all_6b_signal.sh"
cmd = 'parallel -j 4 "python scripts/file_handling/hadd_private.py {}" ::: $(cat sig_files.txt) --eta"'

'parallel -j 4 "python scripts/file_handling/hadd_private.py {}" ::: $(cat sig_files.txt) --eta"'

## Step 2
### B tag SFs and corrections

In [3]:
maxbtag = SixB(get_NMSSM(700,400,selection='maxbtag',private=True,suffix='2M'), feyn=False)
selection = SixB(get_NMSSM(1000,400))

ntuple: /eos/uscms/store/user/srosenzw/sixb/ntuples/Summer2018UL/maxbtag/NMSSM/NMSSM_XYH_YToHH_6b_MX_700_MY_400_2M/ntuple.root
ntuple: /eos/uscms/store/user/srosenzw/sixb/ntuples/Summer2018UL/maxbtag_4b/Official_NMSSM/NMSSM_XToYHTo6B_MX-1000_MY-400_TuneCP5_13TeV-madgraph-pythia8/ntuple.root
model: /eos/uscms/store/user/srosenzw/weaver/models/exp_sixb_official/feynnet_ranker_6b/20230731_7d266883bbfb88fe4e226783a7d1c9db_ranger_lr0.0047_batch2000_withbkg/predict_output/2018/NMSSM_XToYHTo6B_MX-1000_MY-400_TuneCP5_13TeV-madgraph-pythia8.root


In [4]:
assert not ak.all(maxbtag.tree['bSFshape_central'].array() == 1)

In [5]:
cmd = 'parallel -j 4 "python scripts/calculate_2d_sf_corrs.py {}" ::: $(cat filelists/private.txt) --eta'
subprocess.call(cmd, shell=True)

ntuple: /eos/uscms/store/user/srosenzw/sixb/ntuples/Summer2018UL/maxbtag/NMSSM/NMSSM_XYH_YToHH_6b_MX_1000_MY_350/ntuple.root
ntuple: /eos/uscms/store/user/srosenzw/sixb/ntuples/Summer2018UL/maxbtag/NMSSM/NMSSM_XYH_YToHH_6b_MX_1000_MY_300/ntuple.root
ntuple: /eos/uscms/store/user/srosenzw/sixb/ntuples/Summer2018UL/maxbtag/NMSSM/NMSSM_XYH_YToHH_6b_MX_1000_MY_250_2M/ntuple.root
ntuple: /eos/uscms/store/user/srosenzw/sixb/ntuples/Summer2018UL/maxbtag/NMSSM/NMSSM_XYH_YToHH_6b_MX_1000_MY_550/ntuple.root
ntuple: /eos/uscms/store/user/srosenzw/sixb/ntuples/Summer2018UL/maxbtag/NMSSM/NMSSM_XYH_YToHH_6b_MX_1000_MY_450_2M/ntuple.root
ntuple: /eos/uscms/store/user/srosenzw/sixb/ntuples/Summer2018UL/maxbtag/NMSSM/NMSSM_XYH_YToHH_6b_MX_1000_MY_500_2M/ntuple.root
ntuple: /eos/uscms/store/user/srosenzw/sixb/ntuples/Summer2018UL/maxbtag/NMSSM/NMSSM_XYH_YToHH_6b_MX_1000_MY_400_2M/ntuple.root
ntuple: /eos/uscms/store/user/srosenzw/sixb/ntuples/Summer2018UL/maxbtag/NMSSM/NMSSM_XYH_YToHH_6b_MX_1100_MY_250_

usage: calculate_2d_sf_corrs.py [-h] filename
calculate_2d_sf_corrs.py: error: the following arguments are required: filename


ntuple: /eos/uscms/store/user/srosenzw/sixb/ntuples/Summer2018UL/maxbtag/NMSSM/NMSSM_XYH_YToHH_6b_MX_900_MY_700_2M/ntuple.root
ntuple: /eos/uscms/store/user/srosenzw/sixb/ntuples/Summer2018UL/maxbtag/NMSSM/NMSSM_XYH_YToHH_6b_MX_950_MY_700/ntuple.root
ntuple: /eos/uscms/store/user/srosenzw/sixb/ntuples/Summer2018UL/maxbtag/NMSSM/NMSSM_XYH_YToHH_6b_MX_950_MY_800/ntuple.root


1

## Step 3
### FeynNet Predictions & Efficiency Evaluation
FeynNet predictions are done before running this notebook. Efficiencies are evaluated here.

In [8]:
import json
new_model_path = "/eos/uscms/store/user/srosenzw/weaver/cmsuf/data/store/user/srosenzw/lightning/models/feynnet_lightning/X_YH_3H_6b/x3h/lightning_logs/version_23183119/predict/"
with open(f"{new_model_path}/samples.json") as f:
    mass_dict = json.load(f)
mass_dict

import uproot
test_masspoint = 'NMSSM_XToYHTo6B_MX-1100_MY-900_TuneCP5_13TeV-madgraph-pythia8'
f = uproot.open(f"{new_model_path}/{mass_dict[test_masspoint]}.root")
t = f['Events']
t['sorted_rank'].array(), t['sorted_j_assignments'].array()

(<Array [[1.28, 0.763, ..., 0.105, 0.00897], ...] type='21288 * 15 * float32'>,
 <Array [[[0, 1, 2, 5, 3, 4], ..., [0, ...]], ...] type='21288 * 5 * 6 * int64'>)

## Step 4
### Studies
Such as efficiency studies, etc.

## Step 5
### Produce datacards and generate Combine root files containing various MX distributions

In [ ]:
cmd = 'parallel -j 10 "python scripts/generate_combine_root_files.py {}" ::: $(cat sig_files.txt) --eta'
subprocess.call(cmd, shell=True)

## Step 6
### Run HiggsCombine

In [ ]:
import uproot
f = uproot.open